## Шаг 1. Получение данных и изучение информации

В переменной scoring сохраним входные данные с персональной информацией клиентов и условиями договора. Выведем первые 10 строк датасета и изучим информацию о типах данных.

In [1]:
import pandas as pd
scoring = pd.read_csv('/datasets/data.csv')

print(scoring.head(10))
scoring.info()

   children  days_employed  dob_years education  education_id  \
0         1   -8437.673028         42    высшее             0   
1         1   -4024.803754         36   среднее             1   
2         0   -5623.422610         33   Среднее             1   
3         3   -4124.747207         32   среднее             1   
4         0  340266.072047         53   среднее             1   
5         0    -926.185831         27    высшее             0   
6         0   -2879.202052         43    высшее             0   
7         0    -152.779569         50   СРЕДНЕЕ             1   
8         2   -6929.865299         35    ВЫСШЕЕ             0   
9         0   -2188.756445         41   среднее             1   

      family_status  family_status_id gender income_type  debt   total_income  \
0   женат / замужем                 0      F   сотрудник     0  253875.639453   
1   женат / замужем                 0      F   сотрудник     0  112080.014102   
2   женат / замужем                 0    

### Вывод

Входные данные от банка представляют собой массив данных с записями о 21525 клиентах. При просмотре содержимого таблицы выявлены:
- отрицательные значения в столбцах "количество детей в семье" и "общий трудовой стаж в днях"; 
- нулевые значения в столбце "возраст клиента в годах";
- наличие пропущенных данных в столбцах "общий трудовой стаж в годах" и "ежемесячный доход";
- отличающийся регистр в строках столбца "уровень образования клиента".

Вероятно, появление артефактов в данных связано с ошибками, вызванными человеческим фактором, а также технологическими ошибками при выполнении операций с данными (копирование, форматирование, запись данных и т.д). Массив данных требует обработки для дальнейшего исследования влияния семейного положения и количества детей клиента на факт погашения кредита в срок.

## 2. Предобработка данных

### 2.1. Обработка пропусков

In [2]:
# выводим столбцы, где имеются пропущенные значения
print(scoring[scoring['days_employed'].isna()].count())

# заполняем нулями проущенные данные в столбце "общий трудовой стаж в днях"
scoring['days_employed'] = scoring['days_employed'].fillna(0)

# оценка пропущенных значений в столбце "доход" и замена на медианные значения
total_income_median = scoring['total_income'].median()
scoring['total_income'] = scoring['total_income'].fillna(value = total_income_median)

children            2174
days_employed          0
dob_years           2174
education           2174
education_id        2174
family_status       2174
family_status_id    2174
gender              2174
income_type         2174
debt                2174
total_income           0
purpose             2174
dtype: int64


In [3]:
scoring.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


##### Вывод
1. При просмотре таблицы о статистике платежеспособности клиентов обнаружено, что в 2174 записях отсутствуют данные в столбцах:
   - Общий трудовой стаж в днях;
   - Ежемесячный доход.

Можно предположить, что это связано с желанием клиентов воспользоваться каким-либо акционным предложением, например, кредитной программой с минимальным пакетом документов, и получить кредит по упрощенным условиям без предоставления справок о доходах и трудовом стаже. 

2. Основываясь на том, что:
   - Столбцы, в которых имеются пропущенные значения, не интересуют нас для анализа вопросов, указанных в описании проекта;
   - Их доля составляет ~10% от общего числа наблюдений.

Удаление строк с пропущенными значениями может привести к искажению результатов анализа. Поэтому отсутствующие данные в столбце "общий трудовой стаж" были заполнены нулями.

3. Для примерной оценки пропущенных данных в столбце "ежемесячный доход" было выбрано медианное значение. Получено, что разница между медианой и среднеарифметическим значениями составляет ~15%,  в то время как разброс между минимальным и максимальным значениями в столбце составляет 2 порядка. Это позволяет использовать медиану в качестве более предпочтительного варианта заполнения отсутствующих данных.

4. Строка со значением 'XNA' в столбце "пол заемщика" осталась без изменений, так как это никак не влияет на последующий анализ. 

5. Нулевые значения в столбце "возраст клиента в годах" также не обрабатывались, так как данные возраста клиента не участвуют в исследовании их влияния на платежеспособность по кредиту.

### 2.2. Замена типа данных

In [4]:
# замена "-" значений в столбцах "количество детей в семье" и "общий трудовой стаж в днях"
scoring['days_employed'] = scoring['days_employed'].abs()
scoring['children'] = scoring['children'].abs()

# замена вещественного типа данных на целочисленный для столбца "общий трудовой стаж в днях"
scoring['days_employed'] = scoring['days_employed'].astype('int')
scoring.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


### Вывод

В данных столбцов "количество детей в семье" и "общий трудовой стаж в днях" отрицательные значения были заменены на положительные.

Поскольку трудовой стаж исчисляется днями, был выбран метод astype() для замены вещественного типа данных в столбце "общий трудовой стаж" на целочисленный.


### 2.3. Обработка дубликатов

In [5]:
# уникальные значения столбца "уровень образования клиента" и их кол-во
print(scoring['education'].value_counts())

# приведение строк столбца "уровень образования клиента" к нижнему регистру
scoring['education'] = scoring['education'].str.lower()
print(scoring['education'].value_counts())

# количество дубликатов в таблице и их удаление
print(scoring.duplicated().sum())
scoring = scoring.drop_duplicates().reset_index(drop=True) #удаление явных дубликатов

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64
среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64
71


In [6]:
print('Количество дубликатов в данных:', scoring.duplicated().sum())

scoring.info()

Количество дубликатов в данных: 0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 12 columns):
children            21454 non-null int64
days_employed       21454 non-null int64
dob_years           21454 non-null int64
education           21454 non-null object
education_id        21454 non-null int64
family_status       21454 non-null object
family_status_id    21454 non-null int64
gender              21454 non-null object
income_type         21454 non-null object
debt                21454 non-null int64
total_income        21454 non-null float64
purpose             21454 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


### Вывод

На первом этапе обработки дубликатов в столбце "уровень образования клиента" все символы строк были приведены к общему регистру. Далее было проведено удаление дубликатов методом drop_duplicates() с новой индексацией.
Появление дубликатов, вероятно, связано с технологической ошибкой при формировании данных и с человеческим фактором при различной формулировке строк.

### 2.4. Лемматизация

In [7]:
from pymystem3 import Mystem
m = Mystem()

# уникальные значения столбца "цель получения кредита" и их кол-во
print(scoring['purpose'].value_counts())

# выделение лемм из столбца "цель получения кредита"
scoring['purpose_lemmatized'] = scoring['purpose'].apply(m.lemmatize)
scoring['purpose_lemmatized_str'] = scoring['purpose_lemmatized'].apply(' '.join).str.strip()

print(scoring.head())

# список уникальных значений столбца "цель получения кредита" для их дальнейшей категоризации 
print(scoring['purpose_lemmatized_str'].unique())


свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
покупка жилья                             646
жилье                                     646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

### Вывод

Обнаружено, что в столбце "цель получения кредита" имеются категории, отражающие одни и те же цели кредита, но отличающиеся формулировкой. Для выделения основ слова и последующего разделения на основные группы проведена лемматизация строк данного столбца.

### 2.5. Категоризация данных

In [8]:
# просмотр количества уникальных значений в столбце "количество детей в семье"
print(scoring['children'].value_counts())

# замена аномальных значений в столбце "количество детей в семье"
scoring['children'] = scoring['children'].replace(20, 2)

# выделение категорий семей по количеству детей
def children_group(children):
    if 1<= children <= 2:
        return 'малодетные'
    if children > 2:
        return 'многодетные'  
    return 'бездетные'
      
scoring['children_group'] = scoring['children'].apply(children_group)
print(scoring.head(10))

0     14091
1      4855
2      2052
3       330
20       76
4        41
5         9
Name: children, dtype: int64
   children  days_employed  dob_years education  education_id  \
0         1           8437         42    высшее             0   
1         1           4024         36   среднее             1   
2         0           5623         33   среднее             1   
3         3           4124         32   среднее             1   
4         0         340266         53   среднее             1   
5         0            926         27    высшее             0   
6         0           2879         43    высшее             0   
7         0            152         50   среднее             1   
8         2           6929         35    высшее             0   
9         0           2188         41   среднее             1   

      family_status  family_status_id gender income_type  debt   total_income  \
0   женат / замужем                 0      F   сотрудник     0  253875.639453   
1   женат

In [9]:
# выделение категорий по цели получения кредита
def purpose_group(purpose):
    if 'жилье' in purpose:
        return 'операции с недвижимостью'
    if 'недвижимость' in purpose:
        return 'операции с недвижимостью'
    if 'автомобиль' in purpose:
        return 'покупка автомобиля'
    if 'образование' in purpose:
        return 'образование'
    if 'свадьба' in purpose:
        return 'проведение свадьбы'

scoring['purpose_group'] = scoring['purpose_lemmatized_str'].apply(purpose_group)
print(scoring['purpose_group'].value_counts())


операции с недвижимостью    10811
покупка автомобиля           4306
образование                  4013
проведение свадьбы           2324
Name: purpose_group, dtype: int64


In [10]:
# категоризация данных в столбце "ежемесячный доход" 
print(scoring['total_income'].sort_values())
print(scoring['total_income'].count())

# используем функцию qcut, чтобы автоматическими рассчитать 5 квантилей
total_income_group = pd.qcut(scoring['total_income'], 5)

print(total_income_group)

14555    2.066726e+04
12983    2.120528e+04
16138    2.136765e+04
1598     2.169510e+04
14247    2.189561e+04
             ...     
17137    1.711309e+06
20742    1.715018e+06
9159     1.726276e+06
19548    2.200852e+06
12390    2.265604e+06
Name: total_income, Length: 21454, dtype: float64
21454
0        (214606.464, 2265604.029]
1          (98538.268, 135448.912]
2          (135448.912, 156426.09]
3        (214606.464, 2265604.029]
4          (156426.09, 214606.464]
                   ...            
21449    (214606.464, 2265604.029]
21450      (135448.912, 156426.09]
21451       (20667.263, 98538.268]
21452    (214606.464, 2265604.029]
21453       (20667.263, 98538.268]
Name: total_income, Length: 21454, dtype: category
Categories (5, interval[float64]): [(20667.263, 98538.268] < (98538.268, 135448.912] < (135448.912, 156426.09] < (156426.09, 214606.464] < (214606.464, 2265604.029]]


### Вывод

##### 1. Категоризация по количеству детей в семье
На первом этапе были выявлены и заменены артефакты в столбце "количество детей в семье". Было сделано допущение, что значение "20" в столбце связано с ошибкой и заменено на значение "2". 

По наличию и числу детей семьи были разделены на 3 группы:
- бездетные;
- малодетные;
- многодетные.

Данное разделение было основано на предположении, что в большинстве случаев малодетной считается семья с одним-двумя детьми. В соответствии с ГОСТ Р 52495-2005. «Национальный стандарт Российской Федерации. Социальное обслуживание населения. Термины и определения», утвержденный приказом Ростехрегулирования от 30.12.2005 № 532-ст. многодетная семья: семья, имеющая трех и более детей, признанная в установленном порядке многодетной в соответствии с порогом многодетности, установленным органами исполнительной власти субъектов Российской Федерации.

##### 2. Категоризация по цели получения кредита
При просмотре уникальных значений столбца "цели получения кредита" выявлены и разделены цели заемщиков на 4 основные категории:
1. Операции с недвижимостью;
2. Покупка автомобиля;
3. Расходы на образование;
4. Проведение свадьбы.

##### 3. Категоризация по ежемесячному доходу клиента
Данные ежемесячного дохода были разделены на 5 групп, ранжированных по мере возрастания денежных доходов.

## 3. Ответы на поставленные вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [11]:
#print(scoring.head())

scoring_pivot_children = scoring.pivot_table(index='children_group', columns='debt', values='family_status_id', aggfunc='count')
scoring_pivot_children['ratio'] = (scoring_pivot_children[1] / (scoring_pivot_children[0] + scoring_pivot_children[1])).round(3)

print(scoring_pivot_children)

debt                0     1  ratio
children_group                    
бездетные       13028  1063  0.075
малодетные       6336   647  0.093
многодетные       349    31  0.082


### Вывод

Из представленных данных видно, что вероятность задолженности заемщика по кредиту минимальна у семей без детей и составляет **7.5%**, кроме того количество займов значительно превышает остальные категории. Можно предположить, что в семьях без детей нет дополнительных трат, им легче планировать свой бюджет. Тем самым снижается вероятность задолженности и они более активно оформляют кредит.

Категория семей с одним или двумя детьми характеризуется наибольшей вероятностью задолженности по кредиту, **9.3%**. Возможно, такие семьи не имеют много опыта с воспитанием детей и не могут грамотно спланировать свои расходы. И в случае непредвиденных ситуаций, требующих финансовых средств, предпочтут потратить деньги на их урегулирование, а не на возмещение кредита.


- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [12]:
scoring_pivot_family = scoring.pivot_table(index='family_status', columns='debt', values='family_status_id', aggfunc='count')
scoring_pivot_family['ratio'] = (scoring_pivot_family[1] / (scoring_pivot_family[0] + scoring_pivot_family[1])).round(3)
scoring_pivot_family_sort = scoring_pivot_family.sort_values(by='ratio', ascending = False)

print(scoring_pivot_family_sort)

debt                       0    1  ratio
family_status                           
Не женат / не замужем   2536  274  0.098
гражданский брак        3763  388  0.093
женат / замужем        11408  931  0.075
в разводе               1110   85  0.071
вдовец / вдова           896   63  0.066


### Вывод

Найдено, что клиенты банка, никогда не состоящие в зарегистированном браке, с наибольшей вероятностью допускают просрочки по кредиту (не женатых/ не замужем и клиентов в гражданском браке - **9.8%** и **9.3%**, соответственно). В то время как люди в браке, либо ранее состоящие в нём, более организованны и ответственны, вероятность задолженности снижается на **2.3-3.2%**,  Кроме того, люди в браке более активно оформляют кредит, количество займов в данной выборке - **11480** и составляет **~60%** от общего числа обращений.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [13]:
# Для группируемого столбца взяли значение total_income_group = pd.qcut(scoring['total_income'], 5)
# из раздела категоризация данных, чтобы автоматическими поделить уровень дохода на 5 квантилей

scoring_pivot_income = scoring.pivot_table(index=total_income_group, values='debt').round(3)

print(scoring_pivot_income)

                            debt
total_income                    
(20667.263, 98538.268]     0.080
(98538.268, 135448.912]    0.083
(135448.912, 156426.09]    0.087
(156426.09, 214606.464]    0.086
(214606.464, 2265604.029]  0.070


### Вывод

Анализ влияния уровня дохода на выплачиваемость кредита показал, что наиболее состоятельные клиенты с большей вероятностью оплатят кредит вовремя (**7.0%**).

Необходимо отметить, что клиенты выборки с минимальным доходом также характеризуются достаточно низкой вероятностью просрочки (**8.0%**). Предположительно, клиентам с наименьшими доходами приходится более рационально управлять своими денежными средствами по сравнению с клиентами с более высоким доходом. 

- Как разные цели кредита влияют на его возврат в срок?

In [14]:
scoring_pivot_purpose = scoring.pivot_table(index='purpose_group', columns='debt', values='family_status_id', aggfunc='count')
scoring_pivot_purpose['ratio'] = (scoring_pivot_purpose[1] / (scoring_pivot_purpose[0] + scoring_pivot_purpose[1])).round(3)
scoring_pivot_purpose_sort = scoring_pivot_purpose.sort_values(by='ratio', ascending = False)
print(scoring_pivot_purpose_sort)

debt                          0    1  ratio
purpose_group                              
покупка автомобиля         3903  403  0.094
образование                3643  370  0.092
проведение свадьбы         2138  186  0.080
операции с недвижимостью  10029  782  0.072


### Вывод

Наименьшая вероятность задолженности по кредиту (**7.2%**) и в тоже время наибольшее количество обращений наблюдается у клиентов, целью которых является кредит на операции с недвижимостью. Вероятно, это связано с тем, что в большинстве случаев сумма займа для покупки собственного жилья или коммерческой недвижимости значительно превышает запрашиваемую сумму по другим целям кредита. Поэтому потенциальные заемщики, особенно те кто нуждается в жилье, с большой ответственностью подходят к вопросу оформления кредита и его выплаты.

Клиенты, взявшие кредит на проведение свадьбы, характеризуются достаточно высоким процентом выплачиваемости (**8.0%**). Можно предположить, что такой категории заемщиков проще выплатить кредит из подаренных родственниками и друзьями денежных средств.

Практически совпадает и выше вероятность задолженности по кредиту у клиентов, обращающихся в банк с целью покупки автомобиля (**9.4%**) и получения денежных средств на оплату образования (**9.2%**).

## 4. Общий вывод

На основе исследования представленной статистики о платёжеспособности клиентов банка можно сделать следующие предположения:
- Отсутствие детей у заемщика снижает вероятность задолженности по кредиту;
- Клиенты, состоящие в браке ранее или в момент обращения, характеризуются высоким процентом погашаемости кредита в срок;
- Люди с высоким уровнем достатка с большой долей вероятности выплатят оформленный кредит;
- Для клиентов банка с необходимостью покупки недвижимости ответственно подходят к оплает кредита.

Таким образом установлено, что все анализируемые факторы - наличие детей, семейное положение, сумма ежемесячного дохода, цель получения кредита - влияют на факт погашения кредита в срок. Для более детального анализа требуется изучения влияние факторов в совокупности.
